In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as matlab

In [2]:
dataset=pd.read_csv("preprocessed_data.csv")
dataset

,transaction_id,transaction_amount,location,merchant,age,gender,fraud_label
0,1,1000.0,New York,ABC Corp,35,M,0
1,2,500.0,Chicago,XYZ Inc,45,F,0
2,3,2000.0,Los Angeles,ABC Corp,28,M,1
3,4,1500.0,San Francisco,XYZ Inc,30,F,0
4,5,800.0,Chicago,ABC Corp,50,F,0
...,...,...,...,...,...,...,...
81,82,1500.0,Los Angeles,XYZ Inc,31,M,0
82,83,2800.0,San Francisco,ABC Corp,50,F,1
83,84,1350.0,Chicago,XYZ Inc,28,M,0
84,85,920.0,New York,ABC Corp,47,F,0


In [3]:
dataset=dataset.drop(['transaction_id'], axis= 1)

In [4]:
dataset

,transaction_amount,location,merchant,age,gender,fraud_label
0,1000.0,New York,ABC Corp,35,M,0
1,500.0,Chicago,XYZ Inc,45,F,0
2,2000.0,Los Angeles,ABC Corp,28,M,1
3,1500.0,San Francisco,XYZ Inc,30,F,0
4,800.0,Chicago,ABC Corp,50,F,0
...,...,...,...,...,...,...
81,1500.0,Los Angeles,XYZ Inc,31,M,0
82,2800.0,San Francisco,ABC Corp,50,F,1
83,1350.0,Chicago,XYZ Inc,28,M,0
84,920.0,New York,ABC Corp,47,F,0


In [5]:
dataset=pd.get_dummies(dataset,drop_first=True)
dataset

,transaction_amount,age,fraud_label,location_Los Angeles,location_New York,location_San Francisco,merchant_XYZ Inc,gender_M
0,1000.0,35,0,0,1,0,0,1
1,500.0,45,0,0,0,0,1,0
2,2000.0,28,1,1,0,0,0,1
3,1500.0,30,0,0,0,1,1,0
4,800.0,50,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...
81,1500.0,31,0,1,0,0,1,1
82,2800.0,50,1,0,0,1,0,0
83,1350.0,28,0,0,0,0,1,1
84,920.0,47,0,0,1,0,0,0


In [6]:
dataset.columns

Index(['transaction_amount', 'age', 'fraud_label', 'location_Los Angeles',
       'location_New York', 'location_San Francisco', 'merchant_XYZ Inc',
       'gender_M'],
      dtype='object')

In [7]:
dataset["fraud_label"].value_counts()

0    70
1    16
Name: fraud_label, dtype: int64

In [8]:
independent=dataset.drop(['fraud_label'], axis= 1)

In [9]:
independent

,transaction_amount,age,location_Los Angeles,location_New York,location_San Francisco,merchant_XYZ Inc,gender_M
0,1000.0,35,0,1,0,0,1
1,500.0,45,0,0,0,1,0
2,2000.0,28,1,0,0,0,1
3,1500.0,30,0,0,1,1,0
4,800.0,50,0,0,0,0,0
...,...,...,...,...,...,...,...
81,1500.0,31,1,0,0,1,1
82,2800.0,50,0,0,1,0,0
83,1350.0,28,0,0,0,1,1
84,920.0,47,0,1,0,0,0


In [10]:
dependent=dataset[["fraud_label"]]
dependent

,fraud_label
0,0
1,0
2,1
3,0
4,0
...,...
81,0
82,1
83,0
84,0


In [11]:
#split into training set and test
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test=train_test_split(independent,dependent,test_size=1/3,random_state=0)

In [12]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train_= sc.fit_transform(X_train)
X_test_= sc.transform(X_test)

In [13]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import GridSearchCV

param_grid = {'alpha': [1.0, 0.1, 0.01, 0.001]}
grid = GridSearchCV(MultinomialNB(), param_grid, refit = True, verbose = 3,n_jobs=1,scoring='f1_weighted') 
grid.fit(X_train, Y_train)
re=grid.cv_results_
grid_predictions = grid.predict(X_test) 
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(Y_test, grid_predictions)
from sklearn.metrics import classification_report
clf_report = classification_report(Y_test, grid_predictions)             
print(grid.best_params_)
table=pd.DataFrame.from_dict(re)
table
from sklearn.metrics import f1_score
f1_macro=f1_score(Y_test,grid_predictions,average='weighted')
print("The f1_macro value for best parameter {}:".format(grid.best_params_),f1_macro)
print("The confusion Matrix:\n",cm)
print("The report:\n",clf_report)
from sklearn.metrics import roc_auc_score
roc_auc_score(Y_test,grid.predict_proba(X_test)[:,1])

C:\ProgramData\Anaconda3\envs\aiml\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
C:\ProgramData\Anaconda3\envs\aiml\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\ProgramData\Anaconda3\envs\aiml\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\ProgramData\Anaconda3\envs\aiml\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the s

Fitting 3 folds for each of 4 candidates, totalling 12 fits
[CV] alpha=1.0 .......................................................
[CV] ........................... alpha=1.0, score=0.860, total=   0.1s
[CV] alpha=1.0 .......................................................
[CV] ........................... alpha=1.0, score=0.904, total=   0.0s
[CV] alpha=1.0 .......................................................
[CV] ........................... alpha=1.0, score=0.851, total=   0.0s
[CV] alpha=0.1 .......................................................
[CV] ........................... alpha=0.1, score=0.860, total=   0.0s
[CV] alpha=0.1 .......................................................
[CV] ........................... alpha=0.1, score=0.904, total=   0.0s
[CV] alpha=0.1 .......................................................
[CV] ........................... alpha=0.1, score=0.851, total=   0.0s
[CV] alpha=0.01 ......................................................
[CV] ............

C:\ProgramData\Anaconda3\envs\aiml\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\ProgramData\Anaconda3\envs\aiml\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\ProgramData\Anaconda3\envs\aiml\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\ProgramData\Anaconda3\envs\aiml\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please 

0.9545454545454545

In [14]:
from sklearn.naive_bayes import BernoulliNB
from sklearn.model_selection import GridSearchCV

param_grid = { 'alpha': [1.0, 0.1, 0.01, 0.001], 'binarize': [0.0, 0.1, 0.2, 0.3]}
grid = GridSearchCV(BernoulliNB(), param_grid, refit = True, verbose = 3,n_jobs=1,scoring='f1_weighted') 
grid.fit(X_train, Y_train)
re=grid.cv_results_
grid_predictions = grid.predict(X_test) 
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(Y_test, grid_predictions)
from sklearn.metrics import classification_report
clf_report = classification_report(Y_test, grid_predictions)             
print(grid.best_params_)
table=pd.DataFrame.from_dict(re)
table
from sklearn.metrics import f1_score
f1_macro=f1_score(Y_test,grid_predictions,average='weighted')
print("The f1_macro value for best parameter {}:".format(grid.best_params_),f1_macro)
print("The confusion Matrix:\n",cm)
print("The report:\n",clf_report)
from sklearn.metrics import roc_auc_score
roc_auc_score(Y_test,grid.predict_proba(X_test)[:,1])

C:\ProgramData\Anaconda3\envs\aiml\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
C:\ProgramData\Anaconda3\envs\aiml\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\ProgramData\Anaconda3\envs\aiml\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\ProgramData\Anaconda3\envs\aiml\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), fo

Fitting 3 folds for each of 16 candidates, totalling 48 fits
[CV] alpha=1.0, binarize=0.0 .........................................
[CV] ............. alpha=1.0, binarize=0.0, score=0.770, total=   0.0s
[CV] alpha=1.0, binarize=0.0 .........................................
[CV] ............. alpha=1.0, binarize=0.0, score=0.770, total=   0.0s
[CV] alpha=1.0, binarize=0.0 .........................................
[CV] ............. alpha=1.0, binarize=0.0, score=0.770, total=   0.0s
[CV] alpha=1.0, binarize=0.1 .........................................
[CV] ............. alpha=1.0, binarize=0.1, score=0.770, total=   0.0s
[CV] alpha=1.0, binarize=0.1 .........................................
[CV] ............. alpha=1.0, binarize=0.1, score=0.770, total=   0.0s
[CV] alpha=1.0, binarize=0.1 .........................................
[CV] ............. alpha=1.0, binarize=0.1, score=0.770, total=   0.0s
[CV] alpha=1.0, binarize=0.2 .........................................
[CV] ...........

C:\ProgramData\Anaconda3\envs\aiml\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\ProgramData\Anaconda3\envs\aiml\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\ProgramData\Anaconda3\envs\aiml\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\ProgramData\Anaconda3\envs\aiml\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ra

[CV] ............. alpha=1.0, binarize=0.3, score=0.770, total=   0.0s
[CV] alpha=0.1, binarize=0.0 .........................................
[CV] ............. alpha=0.1, binarize=0.0, score=0.770, total=   0.0s
[CV] alpha=0.1, binarize=0.0 .........................................
[CV] ............. alpha=0.1, binarize=0.0, score=0.770, total=   0.0s
[CV] alpha=0.1, binarize=0.0 .........................................
[CV] ............. alpha=0.1, binarize=0.0, score=0.770, total=   0.0s
[CV] alpha=0.1, binarize=0.1 .........................................
[CV] ............. alpha=0.1, binarize=0.1, score=0.770, total=   0.0s
[CV] alpha=0.1, binarize=0.1 .........................................
[CV] ............. alpha=0.1, binarize=0.1, score=0.770, total=   0.0s
[CV] alpha=0.1, binarize=0.1 .........................................
[CV] ............. alpha=0.1, binarize=0.1, score=0.770, total=   0.0s
[CV] alpha=0.1, binarize=0.2 .........................................
[CV] .

C:\ProgramData\Anaconda3\envs\aiml\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\ProgramData\Anaconda3\envs\aiml\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\ProgramData\Anaconda3\envs\aiml\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\ProgramData\Anaconda3\envs\aiml\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'prec

[CV] ............ alpha=0.01, binarize=0.2, score=0.770, total=   0.0s
[CV] alpha=0.01, binarize=0.2 ........................................
[CV] ............ alpha=0.01, binarize=0.2, score=0.770, total=   0.0s
[CV] alpha=0.01, binarize=0.3 ........................................
[CV] ............ alpha=0.01, binarize=0.3, score=0.770, total=   0.0s
[CV] alpha=0.01, binarize=0.3 ........................................
[CV] ............ alpha=0.01, binarize=0.3, score=0.770, total=   0.0s
[CV] alpha=0.01, binarize=0.3 ........................................
[CV] ............ alpha=0.01, binarize=0.3, score=0.770, total=   0.0s
[CV] alpha=0.001, binarize=0.0 .......................................
[CV] ........... alpha=0.001, binarize=0.0, score=0.770, total=   0.0s
[CV] alpha=0.001, binarize=0.0 .......................................
[CV] ........... alpha=0.001, binarize=0.0, score=0.770, total=   0.0s
[CV] alpha=0.001, binarize=0.0 .......................................
[CV] .

C:\ProgramData\Anaconda3\envs\aiml\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\ProgramData\Anaconda3\envs\aiml\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\ProgramData\Anaconda3\envs\aiml\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\ProgramData\Anaconda3\envs\aiml\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'prec

[CV] ........... alpha=0.001, binarize=0.2, score=0.770, total=   0.0s
[CV] alpha=0.001, binarize=0.2 .......................................
[CV] ........... alpha=0.001, binarize=0.2, score=0.770, total=   0.0s
[CV] alpha=0.001, binarize=0.2 .......................................
[CV] ........... alpha=0.001, binarize=0.2, score=0.770, total=   0.0s
[CV] alpha=0.001, binarize=0.3 .......................................
[CV] ........... alpha=0.001, binarize=0.3, score=0.770, total=   0.0s
[CV] alpha=0.001, binarize=0.3 .......................................
[CV] ........... alpha=0.001, binarize=0.3, score=0.770, total=   0.0s
[CV] alpha=0.001, binarize=0.3 .......................................
[CV] ........... alpha=0.001, binarize=0.3, score=0.770, total=   0.0s
{'alpha': 1.0, 'binarize': 0.0}
The f1_macro value for best parameter {'alpha': 1.0, 'binarize': 0.0}: 0.6544962812711291
The confusion Matrix:
 [[22  0]
 [ 7  0]]
The report:
               precision    recall  f1-scor

C:\ProgramData\Anaconda3\envs\aiml\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\ProgramData\Anaconda3\envs\aiml\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\ProgramData\Anaconda3\envs\aiml\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\ProgramData\Anaconda3\envs\aiml\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'prec

0.6883116883116882

In [15]:
from sklearn.naive_bayes import ComplementNB
from sklearn.model_selection import GridSearchCV

param_grid = {'alpha': [1.0, 0.1, 0.01, 0.001]}
grid = GridSearchCV(ComplementNB(), param_grid, refit = True, verbose = 3,n_jobs=1,scoring='f1_weighted') 
grid.fit(X_train, Y_train)
re=grid.cv_results_
grid_predictions = grid.predict(X_test) 
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(Y_test, grid_predictions)
from sklearn.metrics import classification_report
clf_report = classification_report(Y_test, grid_predictions)             
print(grid.best_params_)
table=pd.DataFrame.from_dict(re)
table
from sklearn.metrics import f1_score
f1_macro=f1_score(Y_test,grid_predictions,average='weighted')
print("The f1_macro value for best parameter {}:".format(grid.best_params_),f1_macro)
print("The confusion Matrix:\n",cm)
print("The report:\n",clf_report)
from sklearn.metrics import roc_auc_score

roc_auc_score(Y_test,grid.predict_proba(X_test)[:,1])

C:\ProgramData\Anaconda3\envs\aiml\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
C:\ProgramData\Anaconda3\envs\aiml\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\ProgramData\Anaconda3\envs\aiml\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\ProgramData\Anaconda3\envs\aiml\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the s

Fitting 3 folds for each of 4 candidates, totalling 12 fits
[CV] alpha=1.0 .......................................................
[CV] ........................... alpha=1.0, score=0.817, total=   0.0s
[CV] alpha=1.0 .......................................................
[CV] ........................... alpha=1.0, score=0.817, total=   0.0s
[CV] alpha=1.0 .......................................................
[CV] ........................... alpha=1.0, score=0.904, total=   0.0s
[CV] alpha=0.1 .......................................................
[CV] ........................... alpha=0.1, score=0.817, total=   0.0s
[CV] alpha=0.1 .......................................................
[CV] ........................... alpha=0.1, score=0.860, total=   0.0s
[CV] alpha=0.1 .......................................................
[CV] ........................... alpha=0.1, score=0.904, total=   0.0s
[CV] alpha=0.01 ......................................................
[CV] ............

0.9545454545454545